# Exercise 11 - DIEGO RESTREPO

## Car Price Prediction

Predict if the price of a car is low or high

In [1]:
%matplotlib inline
import pandas as pd
import warnings
warnings.simplefilter("ignore")

data = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTrain_carListings.zip')
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
15,2016,29242,0,0,0,0,1,0,0,1
47,2015,26465,0,0,0,0,1,0,0,1
85,2012,46739,0,1,0,0,0,0,0,1
141,2017,41722,0,0,0,0,0,1,0,1
226,2014,77669,0,0,0,0,0,0,1,0


In [2]:
data.shape

(13150, 10)

In [3]:
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [5]:
import numpy as np

# Exercise 11.1

Estimate a Decision Tree Classifier Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

In [6]:
max_depth = None
num_pct = 10
max_features = None
min_gain=0.001
j = 1
print(X.columns[j])

Mileage


In [7]:
# Split the variable in num_ctp points
#Parte la columna en 10 percentiles
splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / num_pct).tolist())
# Only unique values for filter binary and few unique values features
splits = np.unique(splits)
splits

array([5.00000e+00, 1.58728e+04, 2.32508e+04, 2.98747e+04, 3.56432e+04,
       4.16580e+04, 4.83404e+04, 6.16152e+04, 8.07292e+04, 1.06371e+05])

In [8]:
k=5

In [9]:
filter_l = X.iloc[:, j] < splits[k]

y_l = y.loc[filter_l]
y_r = y.loc[~filter_l]

In [10]:
def gini(y):
    if y.shape[0] == 0:
        return 0
    else:
        return 1 - (y.mean()**2 + (1 - y.mean())**2)

In [11]:
def gini_impurity(X_col, y, split):
    "Calculate the gain of an split k on feature j"
    
    filter_l = X_col < split
    y_l = y.loc[filter_l]
    y_r = y.loc[~filter_l]
    
    n_l = y_l.shape[0]
    n_r = y_r.shape[0]
    
    gini_y = gini(y)
    gini_l = gini(y_l)
    gini_r = gini(y_r)
    
    gini_impurity_ = gini_y - (n_l / (n_l + n_r) * gini_l + n_r / (n_l + n_r) * gini_r)
    
    return gini_impurity_

In [12]:
gini_impurity(X.iloc[:, j], y, splits[k])

0.18496148274516044

In [13]:
def best_split(X, y, num_pct=10):
    
    features = range(X.shape[1])
    
    best_split = [0, 0, 0]  # j, split, gain
    
    # For all features
    for j in features:
        
        splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / (num_pct+1)).tolist())
        splits = np.unique(splits)[1:]
        
        # For all splits
        for split in splits:
            gain = gini_impurity(X.iloc[:, j], y, split)
                        
            if gain > best_split[2]:
                best_split = [j, split, gain]
    
    return best_split

In [14]:
j, split, gain = best_split(X_train, y_train, 5)
j, split, gain

(1, 56598.66666666667, 0.23507164147905313)

In [15]:
filter_l = X.iloc[:, j] < split

y_l = y.loc[filter_l]
y_r = y.loc[~filter_l]

In [16]:
def tree_grow(X, y, level=0, min_gain=0.001, max_depth=None, num_pct=10):
    
    # If only one observation
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=level, split=-1, n_samples=1, gain=0)
        return tree
    
    # Calculate the best split
    j, split, gain = best_split(X, y, num_pct)
    
    # save tree and estimate prediction
    y_pred = int(y.mean() >= 0.5) 
    y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Laplace correction
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0], gain=-1)
    
    # Check stooping criteria
    if gain < min_gain:
        return tree
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # No stooping criteria was meet, then continue to create the partition
    filter_l = X.iloc[:, j] < split
    X_l, y_l = X.loc[filter_l], y.loc[filter_l]
    X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]
    tree['gain'] = [j, gain]
    
    # Next iteration to each split
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    tree['sr'] = tree_grow(X_r, y_r, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    
    return tree

In [17]:
tree = tree_grow(X, y, level=0, min_gain=0.001, max_depth=3, num_pct=10)

In [18]:
tree

{'y_pred': 1,
 'y_prob': 0.5795316301703163,
 'level': 0,
 'split': [1, 51704.54545454545],
 'n_samples': 13150,
 'gain': [1, 0.23348567756020572],
 'sl': {'y_pred': 1,
  'y_prob': 0.8377538829151733,
  'level': 1,
  'split': [0, 2014.0],
  'n_samples': 8368,
  'gain': [0, 0.0359166442135464],
  'sl': {'y_pred': 0,
   'y_prob': 0.3403880070546737,
   'level': 2,
   'split': [0, 2012.0],
   'n_samples': 565,
   'gain': [0, 0.06001982703810749],
   'sl': {'y_pred': 0,
    'y_prob': 0.058823529411764705,
    'level': 3,
    'split': -1,
    'n_samples': 151,
    'gain': -1},
   'sr': {'y_pred': 0,
    'y_prob': 0.44471153846153844,
    'level': 3,
    'split': -1,
    'n_samples': 414,
    'gain': -1}},
  'sr': {'y_pred': 1,
   'y_prob': 0.8737988468930173,
   'level': 2,
   'split': [0, 2015.0],
   'n_samples': 7803,
   'gain': [0, 0.015250286354762527],
   'sl': {'y_pred': 1,
    'y_prob': 0.731399157697707,
    'level': 3,
    'split': -1,
    'n_samples': 2135,
    'gain': -1},
   'sr

In [19]:
def tree_predict(X, tree, proba=False):
    
    predicted = np.ones(X.shape[0])

    # Check if final node
    if tree['split'] == -1:
        if not proba:
            predicted = predicted * tree['y_pred']
        else:
            predicted = predicted * tree['y_prob']
            
    else:
        
        j, split = tree['split']
        filter_l = (X.iloc[:, j] < split)
        X_l = X.loc[filter_l]
        X_r = X.loc[~filter_l]

        if X_l.shape[0] == 0:  # If left node is empty only continue with right
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)
        elif X_r.shape[0] == 0:  # If right node is empty only continue with left
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
        else:
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)

    return predicted    

In [20]:
tree_grow(X, y, level=0, min_gain=0.001, max_depth=1, num_pct=10)

{'y_pred': 1,
 'y_prob': 0.5795316301703163,
 'level': 0,
 'split': [1, 51704.54545454545],
 'n_samples': 13150,
 'gain': [1, 0.23348567756020572],
 'sl': {'y_pred': 1,
  'y_prob': 0.8377538829151733,
  'level': 1,
  'split': -1,
  'n_samples': 8368,
  'gain': -1},
 'sr': {'y_pred': 0,
  'y_prob': 0.12771739130434784,
  'level': 1,
  'split': -1,
  'n_samples': 4782,
  'gain': -1}}

In [21]:
y_pred=tree_predict(X_test, tree)

In [22]:
from sklearn import metrics
print('Accuracy:',metrics.accuracy_score(y_pred, y_test)) 
print("F1-score:",metrics.f1_score(y_pred, y_test))

Accuracy: 0.8619815668202765
F1-score: 0.8833041106565361


# Exercise 11.2

Estimate a Bagging of 10 Decision Tree Classifiers Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

In [23]:
np .random.seed(123)
n_samples = X_train.shape[0]
n_B= 300

# create ten bootstrap samples (will be used to select rows from the DataFrame)
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

In [24]:
n_estimators = 10

np.random.seed(123)
semillas = np.random.randint(1,10000,size=n_estimators)

arboles = {}
for i in range(n_estimators):
    arboles[i]=tree_grow(X_train.iloc[samples[i]], y_train.iloc[samples[i]], max_depth=3)
    

In [25]:
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))

for i in range(n_estimators):
    y_pred_df.iloc[:, i] = tree_predict(X_test, arboles[i])

In [26]:
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators/ 2)).astype(np.int)
print('Accuracy:',metrics.accuracy_score(y_pred, y_test)) 
print("F1-score:",metrics.f1_score(y_pred, y_test))

Accuracy: 0.8670506912442396
F1-score: 0.8915617365156926


# Exercise 11.3

Implement the variable max_features on the Decision Tree Classifier created in 11.1.

Compare the impact in the results by varing the parameter max_features

Evaluate the accuracy on the testing set

In [27]:
def tree_grow(X, y, level=0, min_gain=0.001, max_depth=None, num_pct=10,max_features=8):
    
    # If only one observation
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=level, split=-1, n_samples=1, gain=0, random_index=-1)
        return tree
    
    # Calculate the best split  
    sample_index = np.random.choice(a=X.shape[1], size=max_features, replace=False)
    X_sample=X.iloc[:, sample_index]
    j, split, gain = best_split(X_sample, y, num_pct)
    j=sample_index[j]
    # save tree and estimate prediction
    y_pred = int(y.mean() >= 0.5) 
    y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Laplace correction
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level,  random_index=sample_index,split=-1, n_samples=X.shape[0], gain=-1)
    
    # Check stooping criteria
    if gain < min_gain:
        return tree
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # No stooping criteria was meet, then continue to create the partition
    filter_l = X.iloc[:, j] < split
    X_l, y_l = X.loc[filter_l], y.loc[filter_l]
    X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]
    tree['gain'] = gain
    tree['random_index']=sample_index
    # Next iteration to each split
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct,max_features=max_features)
    tree['sr'] = tree_grow(X_r, y_r, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct,max_features=max_features)
    
    return tree

In [28]:
tree=tree_grow(X_train, y_train, level=0, min_gain=0.001, max_depth=None, num_pct=10, max_features=4)
y_pred=tree_predict(X_test, tree)
print('Accuracy:',metrics.accuracy_score(y_pred, y_test))

Accuracy: 0.8705069124423963


Se puede ver que hay un incremento en el accuracy tras la implementación del árbol de decisión con un Max_features = 4

In [29]:
tree=tree_grow(X_train, y_train, level=0, min_gain=0.001, max_depth=None, num_pct=10, max_features=6)
y_pred=tree_predict(X_test, tree)
print('Accuracy:',metrics.accuracy_score(y_pred, y_test))

Accuracy: 0.8679723502304147


In [30]:
tree

{'y_pred': 1,
 'y_prob': 0.5780753517930095,
 'level': 0,
 'random_index': array([2, 5, 3, 6, 7, 0]),
 'split': [0, 2014.0],
 'n_samples': 8810,
 'gain': 0.2302237574807685,
 'sl': {'y_pred': 0,
  'y_prob': 0.08339247693399574,
  'level': 1,
  'random_index': array([5, 4, 7, 1, 8, 2]),
  'split': [1, 44744.72727272727],
  'n_samples': 2816,
  'gain': 0.025120355275051603,
  'sl': {'y_pred': 0,
   'y_prob': 0.437984496124031,
   'level': 2,
   'random_index': array([7, 4, 1, 3, 5, 8]),
   'split': [8, 1.0],
   'n_samples': 256,
   'gain': 0.029533977052359384,
   'sl': {'y_pred': 0,
    'y_prob': 0.3901345291479821,
    'level': 3,
    'random_index': array([4, 3, 0, 6, 8, 2]),
    'split': [0, 2012.0],
    'n_samples': 221,
    'gain': 0.05893736191557769,
    'sl': {'y_pred': 0,
     'y_prob': 0.10526315789473684,
     'level': 4,
     'random_index': array([8, 0, 4, 2, 5, 3]),
     'split': [2, 1.0],
     'n_samples': 55,
     'gain': 0.06734553325462425,
     'sl': {'y_pred': 0,
   

In [31]:
feature_range = range(1, X_train.shape[1]+1)
# list to store the average Accuracy for each value of max_features
accuracy_scores = []

for feature in feature_range:
    tree=tree_grow(X_train, y_train, level=0, min_gain=0.001, max_depth=None, num_pct=10, max_features=feature)
    y_pred=tree_predict(X_test, tree)
    accuracy_scores.append([feature, metrics.accuracy_score(y_pred, y_test)])

In [32]:
accuracy_s=pd.DataFrame(accuracy_scores, columns=['n_feature', 'Accuracy']) 
max_acc=accuracy_s.loc[accuracy_s['Accuracy'].idxmax()]
print('Max_score:\n',max_acc)

Max_score:
 n_feature    6.000000
Accuracy     0.870737
Name: 5, dtype: float64


La gráfica indica el número de Max_features indicado para obtener el máximo accuracy posible. En este caso, 6 features.

# Exercise 11.4

Estimate a Bagging of 10 Decision Tree Classifiers with `max_features = log(n_features)`

Evaluate the accuracy on the testing set

In [34]:
## Boostrap
np.random.seed(123)
n_samples = X_train.shape[0]
n_B = 10
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]

y_pred_df = pd.DataFrame(index=y_test.index, columns=[list(range(n_B))])
trees = {}

for i in range(10):
    trees[i] = tree_grow(X_train.iloc[samples[i]], y_train.iloc[samples[i]], level=0, min_gain=0.001, max_depth=None, num_pct=10,max_features=int(np.log(X.shape[1])))
    y_pred_df.iloc[:, i] = tree_predict(X_test, trees[i])

## Majority voting ans Accuracy score
y_pred = (y_pred_df.sum(axis=1) >= (n_B / 2)).astype(np.int)
print("F1-score:",metrics.f1_score(y_pred, y_test))
print("Accuracy:",metrics.accuracy_score(y_pred, y_test))

F1-score: 0.8886474741988051
Accuracy: 0.8582949308755761


# Exercise 11.5

Using sklearn, train a RandomForestClassifier

Evaluate the accuracy on the testing set

In [35]:
# use 10-fold cross-validation with each value of max_depth
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, random_state=1, n_jobs=-1)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print("F1-score:",metrics.f1_score(y_pred, y_test))
print("Accuracy:",metrics.accuracy_score(y_pred, y_test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


F1-score: 0.8615751789976134
Accuracy: 0.83963133640553


# Exercise 11.6

Find the best parameters of the RandomForestClassifier (max_depth, max_features, n_estimators)

Evaluate the accuracy on the testing set

Para encontrar el estimador óptimo:

In [36]:
# list of values to try for n_estimators
estimator_range = range(10, 310, 10)
# list to store the average Accuracy for each value of n_estimators
accuracy_s = []
# use 5-fold cross-validation with each value of n_estimators (WARNING: SLOW!)
for estimator in estimator_range:
    clf = RandomForestClassifier(n_estimators=estimator, random_state=1, n_jobs=-1)
    accuracy_s.append([estimator, cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()])

In [37]:
accuracy_s=pd.DataFrame(accuracy_s, columns=['estimator', 'Accuracy']) 
max_acc=accuracy_s.loc[accuracy_s['Accuracy'].idxmax()]
print('Max score:\n',max_acc)

Max score:
 estimator    20.000000
Accuracy      0.839618
Name: 1, dtype: float64


Para encontrar el Max_features óptimo:

In [39]:
# list of values to try for n_estimators
feature_range = range(1, X_train.shape[1]+1)
# list to store the average Accuracy for each value of n_estimators
accuracy_s = []
# use 5-fold cross-validation with each value of n_estimators (WARNING: SLOW!)

for feature in feature_range:
    clf = RandomForestClassifier(max_features=feature, random_state=1, n_jobs=-1)
    accuracy_s.append([feature, cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()])
    
    
accuracy_s=pd.DataFrame(accuracy_s, columns=['n_feature', 'Accuracy']) 
max_acc=accuracy_s.loc[accuracy_s['Accuracy'].idxmax()]
print('Max score:\n',max_acc)

Max score:
 n_feature    9.000000
Accuracy     0.844334
Name: 8, dtype: float64


Para encontrar el depth óptimo:

In [44]:
# list of values to try for n_estimators
max_depth = range(1,20)
# list to store the average Accuracy for each value of n_estimators
accuracy_scores = []
# use 5-fold cross-validation with each value of n_estimators (WARNING: SLOW!)

for depth in max_depth:
    clf = RandomForestClassifier( max_depth=depth, random_state=1, n_jobs=-1)
    accuracy_scores.append([depth, cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()])
    
    
accuracy_scores=pd.DataFrame(accuracy_scores, columns=['depth', 'Accuracy']) 
max_accuracy=accuracy_scores.loc[accuracy_scores['Accuracy'].idxmax()]
print('Max score:\n',max_accuracy)

Max score:
 depth       8.000000
Accuracy    0.881369
Name: 7, dtype: float64


Los parámetros encontrados son:

- estimator: 20
- Max_features: 9
- Depth:8

Con estos parámetros se evalúa el modelo:

In [45]:
clf = RandomForestClassifier(n_estimators=20, max_features=9,max_depth=8, random_state=1, n_jobs=-1)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

print("F1-score:",metrics.f1_score(y_pred, y_test))
print("Accuracy:",metrics.accuracy_score(y_pred, y_test))

F1-score: 0.8969766994030426
Accuracy: 0.8767281105990783


In [46]:
# compute feature importances
feature_cols = X_train.columns
pd.DataFrame({'feature':feature_cols, 'importance':clf.feature_importances_}).sort_values('importance')

,feature,importance
4,M_CamryBase,0.000040
3,M_Camry4dr,0.000401
7,M_CamrySE,0.002215
5,M_CamryL,0.003963
8,M_CamryXLE,0.010509
2,M_Camry,0.010887
6,M_CamryLE,0.018273
0,Year,0.144713
1,Mileage,0.808999
